In [1]:
import numpy as np
import numba
from numba import jit

In [2]:
SQRT_2PI = np.sqrt(2 * np.pi)

@jit(nopython=True, parallel=True)
def gaussians(x, means, widths):
    '''Return the value of gaussian kernels.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    result = np.exp( -0.5 * ((x - means) / widths)**2 ) / widths
    return result / SQRT_2PI / n

In [3]:
means = np.random.uniform(-1, 1, size=1000000)
widths = np.random.uniform(0.1, 0.3, size=1000000)

gaussians(0.4, means, widths)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


array([1.18943852e-06, 1.51551311e-06, 3.83279136e-07, ...,
       5.61922049e-09, 3.25209812e-06, 2.66948676e-08])

In [4]:
gaussians_nothread = jit(nopython=True)(gaussians.py_func)

%timeit gaussians_nothread(0.4, means, widths)
%timeit gaussians(0.4, means, widths)

15.5 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.79 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%timeit gaussians.py_func(0.4, means, widths) # compare to pure NumPy

16.3 ms ± 701 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
@jit(nopython=True, parallel=True)
def kde(x, means, widths):
    '''Return the value of gaussian kernels.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    result = np.exp( -0.5 * ((x - means) / widths)**2 ) / widths
    return result.mean() / SQRT_2PI

kde_nothread = jit(nopython=True)(kde.py_func)

In [7]:
%timeit kde_nothread(0.4, means, widths)
%timeit kde(0.4, means, widths)

13 ms ± 32.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.26 ms ± 649 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
import random

# Serial version
@jit(nopython=True)
def monte_carlo_pi_serial(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

# Parallel version
@jit(nopython=True, parallel=True)
def monte_carlo_pi_parallel(nsamples):
    acc = 0
    # Only change is here
    for i in numba.prange(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [9]:
%time monte_carlo_pi_serial(int(4e8))
%time monte_carlo_pi_parallel(int(4e8))

CPU times: user 5.53 s, sys: 24 ms, total: 5.55 s
Wall time: 5.57 s
CPU times: user 10.2 s, sys: 44.7 ms, total: 10.2 s
Wall time: 1.46 s


3.14168571